<a href="https://colab.research.google.com/github/HuaiyuZhang/DeepLearning/blob/main/openai/gpt_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.6 MB/s eta 0:00:00


In [ ]:
from sklearn.datasets import fetch_20newsgroups
# https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html
import pandas as pd
import openai

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

In [ ]:
print(sports_dataset['data'][10])

From: brifre1@ac.dal.ca
Subject: Re: Goalie masks
Organization: Dalhousie University, Halifax, Nova Scotia, Canada
Lines: 22

In article <93102@hydra.gatech.EDU>, gtd597a@prism.gatech.EDU (Hrivnak) writes:
> 
> 	I'm starting an informal poll on goalie masks. I'd like to know
> who's mask you think looks the best. I've always like Curtis Joseph's
> of the Blues the best. Anyway, send your nominations to me, or post your
> vote here on r.s.h. My e-mail adress is: gtd597a@prism.gatech.edu
> 
> 	Thanks for your time.

I saw a mask once that had drawings of band-aids, presumably for every puck
that goalie stopped with his face/head.  I can't remember who it was or even
if it was NHL (I see quite a few AHL games here).

This is by far the funniest mask I've seen, and for me funny=cool

> 
> -- 
> GO SKINS!    ||"Now for the next question... Does emotional music have quite
> GO BRAVES!   ||   an effect on you?" - Mike Patton, Faith No More 
> GO HORNETS!  ||
> GO CAPITALS! ||Mike Friedman (Hr

In [ ]:
sports_dataset.target_names[sports_dataset['target'][0]]

'rec.sport.baseball'

In [ ]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


In [ ]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

,prompt,completion
0,From: dougb@comm.mot.com (Doug Bank)\nSubject:...,baseball
1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
2,From: rudy@netcom.com (Rudy Wade)\nSubject: Re...,baseball
3,From: monack@helium.gas.uug.arizona.edu (david...,hockey
4,Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....,baseball


In [ ]:
df.to_json("sport2.jsonl", orient='records', lines=True)

In [ ]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

Analyzing...

- Your file contains 1197 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 11 examples that are very long. These are rows: [134, 200, 281, 320, 404, 595, 704, 838, 1113, 1139, 1174]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts e

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'api_key'

In [ ]:

!openai api fine_tunes.create -t "sport2_prepared_train.jsonl" -v "sport2_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " baseball" -m ada


Found potentially duplicated files with name 'sport2_prepared_train.jsonl', purpose 'fine-tune' and size 1519036 bytes
file-mKnl3VekLgAxJ0huuYWWEIBg
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 1.52M/1.52M [00:00<00:00, 1.96Git/s]
Uploaded file from sport2_prepared_train.jsonl: file-nkaN1xfsZtYEDWRpu8vQ5zHc
Found potentially duplicated files with name 'sport2_prepared_valid.jsonl', purpose 'fine-tune' and size 387349 bytes
file-f37be47Ew7Hom5Di6PqVm85F
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 387k/387k [00:00<00:00, 606Mit/s]
Uploaded file from sport2_prepared_valid.jsonl: file-4mqxpLZbATzY9eq2gdMbs3FU
Created fine-tune: ft-rd0LcaqQqJDzjYZnvo5s233A
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-04 01:38:31] Created fine-tune: ft-rd0LcaqQqJDzjYZnvo5s233A



Str

In [ ]:
!openai api fine_tunes.follow -i ft-rd0LcaqQqJDzjYZnvo5s233A

[2023-03-04 01:38:31] Created fine-tune: ft-rd0LcaqQqJDzjYZnvo5s233A

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-rd0LcaqQqJDzjYZnvo5s233A

